In [1]:
import requests as rq
import bs4
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
## load in the hierarchy information
url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi", 
    "modify.1" : "level4",
    "modify.2" : "level3", 
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
## Now load in the subject data
id = 127
subjectData = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv")
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == id)]
subjectData = subjectData[['roi', 'volume']]
## Merge the subject data with the multilevel data
subjectData = pd.merge(subjectData, multilevel_lookup, on = "roi")
subjectData = subjectData.assign(icv = "ICV")
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))


In [3]:
# get source, targets, values for Sankey Diagram 
df = subjectData
labels = pd.concat([df[column] for column in ['icv', 'level1', 'level2']]).unique()
label_index = {label: i for i, label in enumerate(labels)}
sources, targets, values = [], [], []
for i, row in df.iterrows():
    path = row[['icv', 'level1', 'level2']]
    volume = row['comp']
    for j in range(len(path) - 1):
        sources.append(label_index[path[j]])
        targets.append(label_index[path[j + 1]])
        values.append(volume)

#Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(label=labels.tolist()),
    link=dict(source=sources,
              target=targets,
              value=values),
),)
fig.update_layout(width=700, height=1200, title = 'Subject 127 Sankey Diagram (ICV, Level 1, Level 2)')
fig.show()

In [4]:
# get source, targets, values for Sankey Diagram 
df = subjectData
labels = pd.concat([df[column] for column in ['icv', 'level1', 'level2', 'level3']]).unique()
label_index = {label: i for i, label in enumerate(labels)}
sources, targets, values = [], [], []
for i, row in df.iterrows():
    path = row[['icv', 'level1', 'level2', 'level3']]
    volume = row['comp']
    for j in range(len(path) - 1):
        sources.append(label_index[path[j]])
        targets.append(label_index[path[j + 1]])
        values.append(volume)

#Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(label=labels.tolist()),
    link=dict(source=sources,
              target=targets,
              value=values),
),)
fig.update_layout(width=700, height=1200, title = 'Subject 127 Sankey Diagram (ICV, Level 1, Level 2, Level 3)')
fig.show()

In [5]:
fig = px.sunburst(subjectData, path=['icv', 'level1', 'level2', 'level3', 'level4', 'roi'], 
                  values='comp', width=800, height=800)
fig.show()